In [1]:
# import needed
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results

In [5]:
df_netflix = pd.read_csv('originalDataset/netflix_titles.csv')
df_amazon = pd.read_csv('originalDataset/amazon_prime_titles.csv')
print(len(df_netflix))
print(len(df_amazon))

8807
9668


In [6]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [7]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [8]:
df_netflix["date_added"] = pd.to_datetime(df_netflix['date_added']) #turn dates into dates


""" df_netflix['month_added']=df_netflix['date_added'].dt.month
df_netflix['month_name_added']=df_netflix['date_added'].dt.month_name() #add separate column 4 year date month(?)
df_netflix['year_added'] = df_netflix['date_added'].dt.year
 """
df_netflix.head(3)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [14]:
netflix = df_netflix.query("type == 'Movie'") #keep only movies
netflix.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
netflix.head(3)

NameError: name 'df_netflix' is not defined

In [13]:

#QUERY!!!!!!!!!!!!!

'''SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
WHERE
{
  ?film wdt:P31 wd:Q11424 .
  ?film rdfs:label "Zubaan"@en .
  ?film rdfs:label ?film_label .
  FILTER(lang(?film_label) = 'en')
  ?film wdt:P57 ?director . 
  ?director rdfs:label ?director_label .    
  FILTER(lang(?director_label) = 'en')
  ?director wdt:P21 ?dir_gen . 
  ?dir_gen rdfs:label ?dir_gen_label .
  FILTER(lang(?dir_gen_label) = 'en')
  ?film wdt:P750 ?distributor . 
  ?distributor rdfs:label ?distributor_label .
  FILTER(lang(?distributor_label) = 'en')
  ?film wdt:P444 ?review_score .
}'''

'SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score\nWHERE\n{\n  ?film wdt:P31 wd:Q11424 .\n  ?film rdfs:label "Zubaan"@en .\n  ?film rdfs:label ?film_label .\n  FILTER(lang(?film_label) = \'en\')\n  ?film wdt:P57 ?director . \n  ?director rdfs:label ?director_label .    \n  FILTER(lang(?director_label) = \'en\')\n  ?director wdt:P21 ?dir_gen . \n  ?dir_gen rdfs:label ?dir_gen_label .\n  FILTER(lang(?dir_gen_label) = \'en\')\n  ?film wdt:P750 ?distributor . \n  ?distributor rdfs:label ?distributor_label .\n  FILTER(lang(?distributor_label) = \'en\')\n  ?film wdt:P444 ?review_score .\n}'

In [14]:
p = pprint.PrettyPrinter(indent=1)
# funzia piu o mno come una ricerca su wikidata
API_WD = "https://www.wikidata.org/w/api.php"
params = {
    'action': 'wbsearchentities',# <------------ web search from wikidata
    'format': 'json',
    'language': 'en',
    'search': 'Dick Johnson Is Dead' # the query string
}

# query wd API    
r = requests.get(API_WD, params = params).json() 
p.pprint(r)

{'search': [{'concepturi': 'http://www.wikidata.org/entity/Q84366622',
             'description': '2020 documentary film by Kirsten Johnson',
             'display': {'description': {'language': 'en',
                                         'value': '2020 documentary film by '
                                                  'Kirsten Johnson'},
                         'label': {'language': 'en',
                                   'value': 'Dick Johnson Is Dead'}},
             'id': 'Q84366622',
             'label': 'Dick Johnson Is Dead',
             'match': {'language': 'en',
                       'text': 'Dick Johnson Is Dead',
                       'type': 'label'},
             'pageid': 83667587,
             'repository': 'wikidata',
             'title': 'Q84366622',
             'url': '//www.wikidata.org/wiki/Q84366622'}],
 'searchinfo': {'search': 'Dick Johnson Is Dead'},
 'success': 1}


In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)
ssl._create_default_https_context = ssl._create_unverified_context #????? ceracare poi (lezioni daquino)

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" #<---------- Non è l'endpoint, stiamo fetchando della documentazione di qualche tipo

# prepare the query : 10 random triples
my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(my_SPARQL_query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

p.pprint(results)

{'head': {'vars': ['s', 'p', 'o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:29Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:31Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype':

In [3]:
import time
for x in range(6):
  if x == 3:
    time.sleep(5)
    print(x)
  else:
    print(x)


0
1
2
3
4
5


In [17]:
title = list(netflix['title']) #printa una lista di titoli da netflix
#print(title[0:100], title[100:200])
#print(title[0:100], title[100:200])
print(len(title))


6131


In [22]:
#print(film)
ciccio_prova = title[0:20] 
print(ciccio_prova)


['Dick Johnson Is Dead', 'My Little Pony: A New Generation', 'Sankofa', 'The Starling', 'Je Suis Karl', 'Confessions of an Invisible Girl', "Europe's Most Dangerous Man: Otto Skorzeny in Spain", 'Intrusion', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'The Father Who Moves Mountains', 'The Stronghold', 'Birth of the Dragon', 'Jaws']


In [ ]:
""" print("film1 should be []")
print(film) """


p = pprint.PrettyPrinter(indent=1)

def wikidata_reconciliation(query):

    oneundred_ttls = [ciccio_prova[x:x + 100] for x in range(0, len(ciccio_prova), 100)]
    for i in range(0, len(oneundred_ttls)):
        time.sleep(25)
        

        ssl._create_default_https_context = ssl._create_unverified_context
    
    
    # get the endpoint API
        wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        """ print("query = title[0:2]")
        print(query)
        print("film should be []")
        print(film) """

        for x in oneundred_ttls[i]:
            print(x)
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            ?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            ?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')
            ?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')
            ?film wdt:P444 ?review_score .
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()
            response = requests.head(wikidata_endpoint)
            """if response.status_code == 429:
                print("riproviamo")
                time.sleep(int(response.headers["Retry-After"]))"""
            
            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                director.append(results['results']['bindings'][0]['director_label']['value'])
                gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                    
            
    print(len(film))



wikidata_reconciliation(ciccio_prova)

#wikidata_reconciliation(title[100:200])


Dick Johnson Is Dead
My Little Pony: A New Generation
My Little Pony: A New Generation not found
Sankofa
Sankofa not found
The Starling
Je Suis Karl
Je Suis Karl not found
Confessions of an Invisible Girl
Confessions of an Invisible Girl not found
Europe's Most Dangerous Man: Otto Skorzeny in Spain
Europe's Most Dangerous Man: Otto Skorzeny in Spain not found
Intrusion
Intrusion not found
Avvai Shanmughi
Avvai Shanmughi not found
Go! Go! Cory Carson: Chrissy Takes the Wheel
Go! Go! Cory Carson: Chrissy Takes the Wheel not found
Jeans
Jeans not found
Minsara Kanavu
Minsara Kanavu not found
Grown Ups
Dark Skies
Paranoia
Ankahi Kahaniya
Ankahi Kahaniya not found
The Father Who Moves Mountains
The Father Who Moves Mountains not found
The Stronghold
The Stronghold not found
Birth of the Dragon
Jaws
Jaws 2
Jaws 3
Jaws 3 not found
Jaws: The Revenge
My Heroes Were Cowboys
My Heroes Were Cowboys not found
Safe House
Training Day
InuYasha the Movie 2: The Castle Beyond the Looking Glass
InuYasha

HTTPError: HTTP Error 429: Too Many Requests

# New

In [72]:
df_newdata = pd.read_csv('new_data.csv')
df_newdata = df_newdata.loc[:, ~df_newdata.columns.str.contains('^Unnamed')]
df_newdata.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_newdata['title'] = df_newdata['title'].replace({'"':''}, regex=True)

df_newdata.head(2)
#print(len(df_newdata))

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
1,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000


In [73]:
movie_to_find = df_newdata.query("type == 'Movie'") #keep only movies
serie_to_find = df_newdata.query("type == 'TV Show'")


movie_to_find.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
movie_to_find.head(1)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Data,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,1,0,1000


In [74]:
print(len(df_newdata))
print(len(movie_to_find))
print(len(serie_to_find))
print(12214 -12036)

12214
12036
178
178


In [76]:
movie_title = list(movie_to_find['title'])


print(movie_title.index("Gabriel Fluffy Iglesias: One Show Fits All"))
#print(movie_title[2399:])
print(len(movie_title))
print(movie_title[11876:11877])
print(movie_title.index("Dangerous Afternoon"))


2399
12036
['Cover-Up']
11875


In [ ]:
# old query
"""
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')}
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}
             OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            }
"""

In [68]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)

In [ ]:
# "In Your Hands" == error 500

In [69]:
distributor = []
imdbID = []
rottenscore = []


In [66]:
film = [] #facciamo un po' di liste che ci divertiamo
director = []
gender = []


In [67]:
import time
not_found = []

In [8]:

p = pprint.PrettyPrinter(indent=1)
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:
        
        try:

            print(x)
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label 
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}}
            
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                not_found.append(""+x+"")
                print(""+x+" not found")
                
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "director_label" in results['results']['bindings'][0]:
                    director.append(results['results']['bindings'][0]['director_label']['value'])
                else:
                    director.append("no_data")
                if "dir_gen_label" in results['results']['bindings'][0]:
                    gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                else:
                    gender.append("no_data")
                


        except urllib.error.HTTPError as e:
            print("!!!!!HELP!!!!!")
            print(e.headers)
            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            print(x)
            print(error_title)
            print(query[error_title])
            wikidata_reconciliation(query[error_title:])
            
   



wikidata_reconciliation(movie_title[2637:])

NameError: name 'movie_title' is not defined

In [58]:
#print(not_found)

['My Little Pony: A New Generation', 'Je Suis Karl']


In [87]:
from http.client import IncompleteRead

In [94]:

p = pprint.PrettyPrinter(indent=1)
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation1(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?film ?film_label ?distributor ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?film wdt:P345 ?imdbID.}
            OPTIONAL {?film wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")



        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            wikidata_reconciliation1(query[error_title:])
        except IncompleteRead:
            # Oh well, reconnect and keep trucking
            continue
            




wikidata_reconciliation1(movie_title[0:6000])

Cover-Up not found
Constantine And The Cross not found
Clip: Act 5 - Title 5 not found
Clip: Act 5 - Title 3 not found
A Spark in Nothing not found
A Monkey's Tale not found
50 Summers not found
Warning From Space not found
Trapped Child not found
The Host (English Subtitled) not found
The Green Sea not found
The Consequence not found
Parched 2: Hangry not found
Open Five 2 not found
Magari resto (Maybe I'll stay) not found
Made in Chinatown not found
Legend of a Warrior not found
Lance Woods: Undeniable not found
La Cambiale not found
Kept Woman not found
Invasion of the Cubecumbers not found
Iberion not found
I Motorizzati not found
Hyderabad Dreams not found
Hatched not found
Gopyam - The Secret not found
Goodbye Seventies not found
Gaddappana Duniya (Hindi) not found
Bandivan Mi Yhya Sansari not found
A Stormy Night not found
Tomorrow Never Dies (4K UHD) not found
The World is Not Enough (4K UHD) not found
The Lumineers: III not found
The A Team: Making a Scene not found
Quantum Of

In [95]:
print(len(film))

print(len(director))
print(len(gender))
print(len(distributor))
print(len(imdbID))
print(imdbID[-1])
print(len(rottenscore))

6835
6835
6835
15978
15978
tt1289406
15978


In [1]:
dict = {"title": film, "director": director,
        "director gender": gender}

lists_df = pd.DataFrame(dict)

lists_df.head(5)

NameError: name 'film' is not defined

In [96]:
dict1 = {"distributor": distributor, "id": imdbID, "rating score": rottenscore}
lists1_df = pd.DataFrame(dict1)

lists1_df.head(5)

,distributor,id,rating score
0,no_data,tt0108041,8.2/10
1,Netflix,tt5164438,4.2/10
2,Netflix,tt15204288,no_data
3,no_data,tt5563324,4.5/10
4,no_data,tt0139872,no_data


In [76]:
lists_df.to_csv('savelist.csv')


In [98]:
lists1_df.to_csv('savelist2.csv')

In [24]:
print(film)


['Dick Johnson Is Dead', 'The Starling', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Birth of the Dragon', 'Jaws']


# Clean New Dataset

In [2]:
df_movie_title = pd.read_csv('savelist.csv')
df_movie_id = pd.read_csv('savelist2.csv')
df_serie_title = pd.read_csv('series_director.csv')
df_serie_id = pd.read_csv('series.csv')


In [11]:
df_movie_title = df_movie_title.loc[:, ~df_movie_title.columns.str.contains('^Unnamed')]
df_movie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

df_movie_id = df_movie_id.loc[:, ~df_movie_id.columns.str.contains('^Unnamed')]
df_movie_id.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

df_serie_title = df_serie_title.loc[:, ~df_serie_title.columns.str.contains('^Unnamed')]
df_serie_title.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

df_serie_id = df_serie_id.loc[:, ~df_serie_id.columns.str.contains('^Unnamed')]
df_serie_id.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')

In [28]:
df_movie_title.head(7)
#df_movie_title.info()
#print(len(df_movie_title))

,title,director,director gender
0,Sankofa,Haile Gerima,male
1,The Starling,Theodore Melfi,male
2,Confessions of an Invisible Girl,no_data,no_data
3,Intrusion,no_data,no_data
4,Avvai Shanmughi,K. S. Ravikumar,male
5,Jeans,Nicolette Krebitz,female
6,Minsara Kanavu,Rajiv Menon,male


In [51]:
match_titles = df_movie_title['title'].unique()
print(len(match_titles))

6677


In [52]:
match_ids = df_movie_id['id'].unique()
print(len(match_ids))

6477


In [62]:
ids = df_movie_id['id'].tolist()
print(ids)

    


['tt0108041', 'tt5164438', 'tt15204288', 'tt5563324', 'tt0139872', 'tt0279885', 'tt0282003', 'tt1375670', 'tt2387433', 'tt1413495', 'tt15250998', 'no_data', 'tt5238240', 'tt2720826', 'tt0073195', 'tt0077766', 'tt0093300', 'tt1599348', 'tt0139654', 'tt10521144', 'tt10322274', 'tt8329618', 'no_data', 'tt7737528', 'tt0468536', 'tt15095938', 'tt6127004', 'tt5691024', 'tt10168094', 'tt6167894', 'tt8009744', 'tt11742798', 'tt0109134', 'tt0810784', 'tt4088588', 'tt4699202', 'tt1118045', 'tt0199626', 'tt10930586', 'tt0356470', 'tt0350261', 'tt1823672', 'tt0109444', 'tt0106582', 'tt0159365', 'tt0231402', 'tt0989757', 'tt0097216', 'tt8936646', 'tt0463998', 'tt1133985', 'tt0099800', 'tt0102065', 'tt0110064', 'tt0131436', 'tt0099834', 'tt0439630', 'tt4841640', 'tt0091369', 'tt0892318', 'tt4006302', 'tt0337592', 'tt1602479', 'tt0116996', 'tt5301662', 'tt0270980', 'tt0132347', 'tt0087843', 'tt0409182', 'tt0120014', 'tt1230168', 'tt0332379', 'tt5073746', 'tt0314353', 'tt0041190', 'tt0091129', 'tt0054

In [65]:
unique_ids = []
for x in range(len(ids)):
    if ids[x] == "no_data":
        unique_ids.append(ids[x])
    if ids[x] != "no_data" and ids[x] not in unique_ids:
        unique_ids.append(ids[x])
print(len(unique_ids))

7038


In [35]:

df_movie_title_copy = df_movie_title.drop_duplicates(keep='first')
print(len(df_movie_title_copy))


6688


In [34]:
df_movie_title_copy.duplicated(subset=['title'])
pd.concat(g for _, g in df_movie_title_copy.groupby("title") if len(g) > 1)

,title,director,director gender
2410,Abhinetri,Subodh Mukherjee,male
5692,Abhinetri,Satish Pradhan,male
322,Bliss,Abdullah Oğuz,male
6057,Bliss,Doris Dörrie,female
924,Boss,R. Raghuraj,male
4464,Boss,V. N. Aditya,male
6774,Jack,Francis Ford Coppola,male
6827,Jack,Elisabeth Scharang,female
1594,Lionheart,Genevieve Nnaji,female
5071,Lionheart,Sheldon Lettich,male


In [13]:
df_movie_id.head(7)

,distributor,id,rating score
0,no_data,tt0108041,8.2/10
1,Netflix,tt5164438,4.2/10
2,Netflix,tt15204288,no_data
3,no_data,tt5563324,4.5/10
4,no_data,tt0139872,no_data
5,no_data,tt0279885,no_data
6,AVM Productions,tt0282003,no_data


In [36]:
df_movie_id_copy = df_movie_id.drop_duplicates(keep='first')
print(len(df_movie_id_copy))


6585


In [47]:
singular_value =pd.concat(g for _, g in df_movie_id_copy.groupby("id") if len(g) == 1)
singular_value 


,distributor,id,rating score
644,no_data,nm10640262,no_data
3792,no_data,nm7189780,no_data
9388,no_data,tt0000487,no_data
4530,no_data,tt0001110,no_data
5336,Paramount Pictures,tt0004816,no_data
...,...,...,...
220,no_data,tt9866736,no_data
1031,no_data,tt9877170,no_data
550,no_data,tt9893250,7/10
1094,Netflix,tt9898858,3.8/10


In [43]:

pd.concat(g for _, g in df_movie_id_copy.groupby("id") if len(g) > 1)

,distributor,id,rating score
11,no_data,no_data,no_data
615,Netflix,no_data,no_data
1042,Falcon Pictures,no_data,no_data
1157,YouTube,no_data,no_data
1675,Multivision Plus,no_data,no_data
...,...,...,...
7099,Relativity Media,tt2918436,4/10
6508,Allfilm,tt2991224,7.4/10
12528,iTunes,tt2991224,7.4/10
6484,Netflix,tt8820590,no_data


In [14]:
df_serie_title.head(7)

,title,director,gender
0,Ganglands,no_data,no_data
1,Midnight Mass,Mike Flanagan,male
2,The Great British Baking Show,no_data,no_data
3,The Smart Money Woman,no_data,no_data
4,King of Boys: The Return of the King,Kemi Adetiba,female
5,Valeria,Nely Reguera,female
6,I Need Romance,no_data,no_data


In [15]:
df_serie_id.head(7)

,serie,brodcaster,imdbID,rottenscore
0,Ganglands,no_data,no_data,no_data
1,Midnight Mass,Netflix,tt10574558,no_data
2,The Great British Baking Show,BBC One,tt1877368,no_data
3,The Smart Money Woman,Netflix,no_data,no_data
4,King of Boys: The Return of the King,Netflix,no_data,no_data
5,Valeria,Univision,tt1483311,no_data
6,I Need Romance,tvN,tt3340752,no_data
